In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df

In [ ]:
df['연도'] = df['발생일시'].str.slice(0, 4)


In [ ]:
df['연도'].value_counts()

In [ ]:
df2 = df[df['연도'] == '2023']

In [ ]:
df2['인적사고'].value_counts()

In [ ]:
df2.to_csv('2023.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

In [ ]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
train = pd.read_csv('2023.csv')
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [ ]:
grouped = train.groupby("인적사고")

In [ ]:
res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

In [ ]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

In [ ]:
res


In [ ]:
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

In [ ]:
for i in range(len(test)):
    accident = test.loc[i, "인적사고"]
    sample.loc[i, "재발방지대책 및 향후조치계획"] = res[accident]
    sample.iloc[i, 2:] = res_v[accident]

In [ ]:
sample.to_csv("대표문장.csv", index=False, encoding='utf-8-sig')

In [ ]:
import stanza

# 한국어 모델 다운로드 (최초 1회 실행)
stanza.download('ko')

# 한국어 파이프라인 구성: 토큰화, 품사 태깅, 의존 구문 분석 포함
nlp = stanza.Pipeline('ko')

# 분석할 문장 예시
text = "재발방지대책 및 향후조치계획에서 인적사고별 문구를 추출합니다."
doc = nlp(text)

# 각 문장에 대해 의존 구문 구조 출력
for sentence in doc.sentences:
    for word in sentence.words:
        print(f"단어: {word.text}\t의존관계: {word.deprel}\t헤드 단어 인덱스: {word.head}")


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [ ]:
s1 = '시스템 비계 발판 적재 하중 준수 및 안전교육 실시와 조치계획 이행 확인 통한 재발 방지 대책 및 향후 조치 계획.'
s2 = ' TBM 시 사고사례 전파 및 작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조치 계획'

pred_embed = model.encode(s2)
gt_embed = model.encode(s1)
cosine_similarity(gt_embed,pred_embed)